In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
from tvemoves_rufbad.simulation import Simulation, SimulationParams
from tvemoves_rufbad.domain import RectangleDomain
from tvemoves_rufbad.helpers import (
    austenite_potential,
    martensite_potential,
    total_elastic_potential,
)
from tvemoves_rufbad.tensors import Matrix
import numpy as np
from matplotlib import pyplot as plt
import math

### Austenite and Martensite potentials

In [ ]:
xs = np.linspace(-3.0, 3.0, 500)
aust = np.array([austenite_potential(Matrix([[1, x], [0, 1]])) for x in xs])
mart = np.array([martensite_potential(Matrix([[1, x], [0, 1]])) for x in xs])
plt.plot(xs, aust, color='blue')
plt.plot(xs, mart, color='red')

### Total elastic energy

In [ ]:
xs = np.linspace(-1.0, 1.0, 500)

def total(theta):
    return np.array([total_elastic_potential(Matrix([[1, x], [0, 1]]), theta) for x in xs])
plt.plot(xs, total(0), color='blue')
plt.plot(xs, total(0.5), color='violet')
plt.plot(xs, total(1), color='green')
plt.plot(xs, total(2), color='orange')
plt.plot(xs, total(100), color='red')

In [ ]:
fps = 1
tau = 1 / fps
eps = 1 / 10
params = SimulationParams(initial_temperature=0.0, search_radius=100.0, fps=fps, scale=eps)
domain = RectangleDomain(width=1.0, height=2.0, fix=["lower"])

num_stress_steps = 5
stress_end = num_stress_steps * tau
num_steps = 15


def external_temperature(t: float):
    if t < stress_end:
        return 0.0
    return 10.0


def boundary_traction(t: float, x: float, y: float):
    if t < stress_end and y > 1.0 - eps / 4:
        return [0.0, 4.0 * math.exp(-20 * (x - 0.5) ** 2)]
    return [0.0, 0.0]


sim = Simulation(domain, params, external_temperature, boundary_traction)

In [ ]:
sim.run(num_steps=num_steps - 2)

In [ ]:
sim.max_temp()

In [ ]:
for i in range(len(sim.steps)):
    sim.plot_step(i)